# Text Search Pipeline

Query for documents or indexes of word occurrences inside documents.

## Initial Setup

Import widgets.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

Import other required modules.

In [ ]:
import bs4, csv,json, multiprocessing as mp, numpy as np, os, pandas as pd, pickle, re, requests, tqdm
from inspect import signature
from dotenv import load_dotenv
from pprint import pprint
from termcolor import colored, cprint

Use caching.

In [ ]:
import ipycache
from IPython.utils.traitlets import Unicode
%load_ext ipycache

Configurations.

In [ ]:
# print coloring options
line_color = 'green'
text_color = 'magenta'
reference_color = 'white'

Load environment variables from .env file.

In [ ]:
load_dotenv()
metadata_path=os.getenv(key='metadata_path')
metadata_df_path=os.getenv(key='metadata_df_path')
corpus_path=os.getenv(key='corpus_path')

cprint(text='-' * 100, color=line_color)
cprint(text=f'Metadata path: {metadata_path}', color=text_color)
cprint(text=f'Metadata dataframe path: {metadata_df_path}', color=text_color)
cprint(text=f'Corpus path: {corpus_path}', color=text_color)

## Load Metadata

Load metadata for First1KGreek project.

In [ ]:
metadata_df = None

try:
    metadata_df = pickle.load(file=open(file='metadata_df.pkl', mode='rb'))['metadata_df']

except:
    metadata_csv_reader = csv.reader(open(file=metadata_path, mode='r', encoding='utf-8'))
    columns = next(metadata_csv_reader)
    metadata_df = pd.DataFrame(
        data=np.asarray(a=list(metadata_csv_reader)),
        columns=columns
    )

cprint(text='-' * 100, color=line_color)
cprint(text='metadata_df:', color=text_color)
cprint(text='-' * 100, color=line_color)

In [ ]:
metadata_df

In [ ]:
%%cache metadata_df.pkl metadata_df --force
metadata_df

## Import Data

Import Latin text.

### *From TXT File*

In [ ]:
def load_txt(filename: str):
    """
    Extract text from a .txt file.

    Parameters:
        filename (str): Path of file to load.

    Returns:
        str: Text loaded from file.
    """
    if not os.path.exists(path=filename):
        raise ValueError(f'The path {filename} does not exist.')

    text = open(file=filename, mode='r').read()
    return text

filename = '/mnt/d/share/Using-AI-to-Trace-the-History-of-Race-and-Inequality/src/sample_text/latin/urn_cts_greekLit_stoa0146d.stoa001.opp-lat11.txt'
text = load_txt(filename=filename)
cprint(text='-' * 100, color=line_color)
cprint(text='Loading from TXT file:', color=text_color)
cprint(text=filename, color=reference_color)
cprint(text='-' * 100, color=line_color)
cprint(text=text, color=text_color)

### *From URI*

In [ ]:
def load_uri(uri: str):
    """
    Load text from URI.

    Parameters:
        uri (str): URI link to text online.

    Returns:
        str: Text loaded from URI.
    """
    req = requests.get(url=uri)
    return req.text

uri = 'https://scaife.perseus.org/library/passage/urn:cts:greekLit:stoa0146d.stoa001.opp-lat1:1/text/'
text = load_uri(uri=uri)
cprint(text='-' * 100, color=line_color)
cprint(text='Loading from URI:', color=text_color)
cprint(text=uri, color=reference_color)
cprint(text='-' * 100, color=line_color)
cprint(text=text, color=text_color)

### *From URN*

In [ ]:
def load_urn(urn: str):
    """
    Load text from URN.

    Parameters:
        urn (str): URN link that identifies specific work.

    Returns:
        str: Text loaded from URI.
    """
    idx = metadata_df.index[metadata_df['URN'] == urn][0]
    url = f'https://scaife.perseus.org/library/passage/{metadata_df.at[idx, "URL"].split("/")[-2]}/text/'
    req = requests.get(url=url)
    
    return req.text

urn = 'urn:cts:greekLit:stoa0146d.stoa001.opp-lat1'
text = load_urn(urn=urn)
url = f'https://scaife.perseus.org/library/passage/{metadata_df.at[0, "URL"].split("/")[-2]}/text/'
cprint(text='-' * 100, color=line_color)
cprint(text='Loading from URN:', color=text_color)
cprint(text=urn, color=reference_color)
cprint(text=url, color=text_color)
cprint(text='-' * 100, color=line_color)
cprint(text=text, color=text_color)

Create analyzer for text content.

In [ ]:
from whoosh.lang import has_stemmer, languages
from whoosh.analysis import SpaceSeparatedTokenizer, LowercaseFilter

In [ ]:
# check for Latin availability for analyzer
cprint(text='-' * 100, color=line_color)
cprint(text='Available languages for LanguageAnalyzer:', color=text_color)
cprint(text='-' * 100, color=line_color)
pprint(languages)

cprint(text='-' * 100, color=line_color)
cprint(text=f'Latin is available? {has_stemmer(lang="lat")}', color=text_color)

In [ ]:
analyzer = SpaceSeparatedTokenizer() | LowercaseFilter()
for token in analyzer(value=text):
    pprint(token)

Create schema.

In [ ]:
from whoosh.index import create_in, exists_in, open_dir
from whoosh.fields import Schema, ID, KEYWORD, NUMERIC, TEXT
from whoosh.formats import Positions
from whoosh.qparser import QueryParser

In [ ]:
schema_upload = widgets.FileUpload(
    accept='.json, .pkl, .txt',
    multiple=False,
)

display(schema_upload)

In [ ]:
schema_bytestr = list(schema_upload.value.values())[0]['content']
schema_dict = json.loads(s=schema_bytestr)
for key, val in schema_dict.items():
    schema_dict[key] = eval(val)

cprint(text='-' * 100, color=line_color)
cprint(text='Dictionary form of uploaded schema:', color=text_color)
cprint(text='-' * 100, color=line_color)
pprint(schema_dict)

In [ ]:
Schema(**schema_dict)

In [ ]:
schema = Schema(
    index = NUMERIC(
        bits=64,
        numtype='int',
        signed=False,
        sortable=True,
        stored=True,
        unique=True
    ),
#     url = ID(
#         sortable=False,
#         stored=True,
#         unique=True
#     ),
#     title=TEXT(
#         chars=True,
#         phrase=True,
#         sortable=True,
#         stored=True
#     ),
#     author=TEXT(
#         chars=True,
#         phrase=True,
#         sortable=True,
#         stored=True
#     ),
#     languages=KEYWORD(
#         commas=True,
#         lowercase=True,
#         scorable=False,
#         sortable=False,
#         stored=True,
#         unique=False
#     ),
#     keywords=KEYWORD(
#         commas=True,
#         lowercase=True,
#         scorable=True,
#         sortable=True,
#         stored=True,
#         unique=False
#     ),
    content=TEXT(  
        analyzer=analyzer,
        chars=True,
        phrase=True,
        sortable=False,
        stored=False,

    )
)

Create or load index.

In [ ]:
dirname_dropdown = widgets.Dropdown(
    options=np.asarray(a=list(filter(lambda x: os.path.isdir(s=x), os.listdir(path='.')))),
    description='dirname',
    disabled=False
)

display(dirname_dropdown)

In [ ]:
writelock_files = list(filter(
    lambda x: re.match(pattern='.+_WRITELOCK', string=x), 
    list([x.decode() for x in os.listdir(path=dirname_dropdown.value)]))
)

indexname_options = np.asarray(a=list(map(
    lambda x: x[:-len('_WRITELOCK')],
    writelock_files))
)

indexname_dropdown = widgets.Dropdown(
    options=indexname_options,
    description='indexname',
    disabled=False
)

display(indexname_dropdown)

In [ ]:
ix = None

try:
    ix = open_dir(
        dirname=str(dirname_dropdown.value),
        indexname=indexname_dropdown.value
    )

except:
    raise Exception
    ix = create_in(dirname='indexes/',
                  schema=schema,
                  indexname='latin_index')

Index documents.

In [ ]:
cprint(text='-' * 100, color=line_color)
cprint(text='Enter number of processors for indexing:', color=text_color)
cprint(text='-' * 100, color=line_color)
procs_select = widgets.IntSlider(
    value=64,
    min=1,
    max=128,
    step=1,
    description='Number of processors to use during indexing.',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(procs_select)

In [ ]:
for idx, row in enumerate(tqdm.tqdm(list([row[1] for row in list(metadata_df.iterrows())]))):
    writer = ix.writer()
    writer.add_document(
        index = idx,
#         url = row['URL'],
#         title = row['Work'],
#         author = row['Workgroup'],
#         languages = row['Language'],
#         keywords = u'',
        content = load_urn(urn=row['URN'])
    )

    writer.commit()

Documentation example.

In [ ]:
latin_queries = list([query.strip() for query in open(file='latin_queries.txt').readlines()])
pprint(latin_queries)

In [ ]:
with ix.searcher() as searcher:
    query = QueryParser(
        fieldname='index',
        schema=ix.schema
    ).parse(
#         text=' '.join(latin_queries),
        text='*'
    )
    results = searcher.search(q=query)
    pprint(list(results))
#     pprint(signature(obj=query))